<a href="https://colab.research.google.com/github/Dhanujaya2000/dhanujayaprojects/blob/main/ME421_ControlSystems_Lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Instructions


* Make a copy of this and save it in your github repository.

* Do all your work, EXCLUSIVELY, in that saved notebook. Your github commits will serve as a refelection of your individual contributions. It will constitute 25% of your final grade for the lab.

* Complete Tasks#1 and Task#2 by 5:00PM on 14/02/2024

* Complete Tasks#3 and the Task#4 by 5:00PM on 28/02/2024 and demonstrate the work in the class on 28/02/2024.

* Complete Task#5 by 5:00PM on 14/03/2024 and submit the link to your working repository by midnight 14/03/2024.

#References

* https://colab.research.google.com/github/mugalan/classical-mechanics-from-a-geometric-point-of-view/blob/main/rigid-body-control/Mugas_Rigid_Body_Control.ipynb

* https://github.com/mugalan/classical-mechanics-from-a-geometric-point-of-view/blob/main/rigid-body-control/MaithriPalaDHS_RigidBodyPIDControl_IITB_18_03_2023.pdf

#Task#1

Conside a rigid body moving in space. Let $\mathbf{b}$ be a body fixed frame with the origin coinciding with that of the center of mass of the body, $M$ be the tatal mass of the body, $\mathbb{I}$ be the inertia tensor of the body with respect to the body frame $\mathbf{b}$, $f^e$ be the resultant total external interactions acting on the particles of the body, and $\tau^e$ be the total resultant of the moments of the external interactions about the center of mass of the body.

Show that a general rigid body is descibed by the following simple equations in an inertial frame $\mathbf{e}$ where the moments are taken about the center of mass of the rigid body.

\begin{align}
\dot{o}&=\frac{1}{M}p\\
\dot{R}&=\widehat{\omega}R,\\
\dot{p}&=f^e+f^u,\\
\dot{\pi}&=\tau^e+\tau^u,
\end{align}
where
\begin{align}
\omega &=(\mathbb{I}_c^R)^{-1}\pi,
\end{align}
Here we have split the force and control moments into unmanipulatable and manipulatable (control) part. The manipulatable (control) part will be denoted by a superscript $u$.

- $ o $: Position of the center of mass (CoM).
- $ M $: Total mass of the rigid body.
- $ \mathbb{I} $: Inertia tensor in the body frame.
- $ R $: Rotation matrix from the body frame to the inertial frame.
- $ \omega $: Angular velocity in the body frame.
- $ p = M \dot{o} $: Linear momentum.
- $ \pi = \mathbb{I} \omega $: Angular momentum.
- $ f^e $: External force acting on the rigid body.
- $ \tau^e $: External moment about the CoM.
- $ f^u, \tau^u $: Control force and moment.


$$
M \ddot{o} = f^e + f^u
$$
Substituting $ p = M \dot{o} $:
$$
\dot{o} = \frac{1}{M} p
$$
$$
\dot{p} = f^e + f^u
$$


$$
\frac{d}{dt} (\mathbb{I} \omega) = \tau^e + \tau^u
$$
Substituting $ \pi = \mathbb{I} \omega $:
$$
\dot{\pi} = \tau^e + \tau^u
$$


$$
\dot{R} = \widehat{\omega} R
$$
where $ \widehat{\omega} $ is the skew-symmetric matrix of $ \omega $.


$$
\omega = (\mathbb{I}_c^R)^{-1} \pi
$$

### **Final Set of Equations**
$$
\dot{o} = \frac{1}{M} p
$$
$$
\dot{R} = \widehat{\omega} R
$$
$$
\dot{p} = f^e + f^u
$$
$$
\dot{\pi} = \tau^e + \tau^u
$$
$$
\omega = (\mathbb{I}_c^R)^{-1} \pi
$$

#Task#2

Simulate the motion of a quadrotor UAV without resorting any parameterization of $SO(3)$ such as Euler angles or quaternions.

In [10]:
import numpy as np
import plotly.graph_objects as go
from scipy.integrate import solve_ivp

# Constants
g = 9.81  # Gravity (m/s^2)
m = 1.0   # Mass of the quadrotor (kg)
I = np.diag([0.01, 0.01, 0.02])  # Moment of inertia (kg*m^2)

# Initial conditions
position = np.array([0.0, 0.0, 0.0])  # Initial position (m)
velocity = np.array([0.0, 0.0, 0.0])  # Initial velocity (m/s)
R = np.eye(3)  # Initial orientation (identity matrix, no rotation)
omega = np.array([0.0, 0.0, 0.0])  # Initial angular velocity (rad/s)

# Control inputs (thrust and torques)
def control_inputs(t):
    f_total = m * g + 0.1 * np.sin(t)  # Total thrust (N)
    tau = np.array([0.1 * np.sin(t), 0.1 * np.cos(t), 0.05 * np.sin(t)])  # Torques (N*m)
    return f_total, tau

# Equations of motion
def quadrotor_dynamics(t, state):
    position = state[0:3]
    velocity = state[3:6]
    R = state[6:15].reshape(3, 3)
    omega = state[15:18]

    f_total, tau = control_inputs(t)

    R_dot = R @ np.array([[0, -omega[2], omega[1]],
                          [omega[2], 0, -omega[0]],
                          [-omega[1], omega[0], 0]])

    acceleration = np.array([0, 0, -g]) + (R @ np.array([0, 0, f_total])) / m
    omega_dot = np.linalg.inv(I) @ (tau - np.cross(omega, I @ omega))

    state_dot = np.zeros(18)
    state_dot[0:3] = velocity
    state_dot[3:6] = acceleration
    state_dot[6:15] = R_dot.reshape(9)
    state_dot[15:18] = omega_dot

    return state_dot

# Initial state vector
initial_state = np.concatenate([position, velocity, R.reshape(9), omega])

# Time span for the simulation
t_span = (0, 10)
t_eval = np.linspace(t_span[0], t_span[1], 1000)

# Solve the ODE
sol = solve_ivp(quadrotor_dynamics, t_span, initial_state, t_eval=t_eval)

# Extract the results
position = sol.y[0:3, :]
velocity = sol.y[3:6, :]
omega = sol.y[15:18, :]

# Create Plotly figures
fig_position = go.Figure()
fig_position.add_trace(go.Scatter(x=sol.t, y=position[0], mode='lines', name='x'))
fig_position.add_trace(go.Scatter(x=sol.t, y=position[1], mode='lines', name='y'))
fig_position.add_trace(go.Scatter(x=sol.t, y=position[2], mode='lines', name='z'))
fig_position.update_layout(title='Position Over Time', xaxis_title='Time (s)', yaxis_title='Position (m)')

fig_velocity = go.Figure()
fig_velocity.add_trace(go.Scatter(x=sol.t, y=velocity[0], mode='lines', name='vx'))
fig_velocity.add_trace(go.Scatter(x=sol.t, y=velocity[1], mode='lines', name='vy'))
fig_velocity.add_trace(go.Scatter(x=sol.t, y=velocity[2], mode='lines', name='vz'))
fig_velocity.update_layout(title='Velocity Over Time', xaxis_title='Time (s)', yaxis_title='Velocity (m/s)')

fig_omega = go.Figure()
fig_omega.add_trace(go.Scatter(x=sol.t, y=omega[0], mode='lines', name='ωx'))
fig_omega.add_trace(go.Scatter(x=sol.t, y=omega[1], mode='lines', name='ωy'))
fig_omega.add_trace(go.Scatter(x=sol.t, y=omega[2], mode='lines', name='ωz'))
fig_omega.update_layout(title='Angular Velocity Over Time', xaxis_title='Time (s)', yaxis_title='Angular Velocity (rad/s)')

# Show plots
fig_position.show()
fig_velocity.show()
fig_omega.show()


In [9]:
import numpy as np
import plotly.graph_objects as go
from scipy.integrate import solve_ivp

# Constants
M = 1.0  # Mass of quadrotor (kg)
I = np.diag([0.02, 0.02, 0.04])  # Inertia tensor
g = 9.81  # Gravity (m/s^2)
e3 = np.array([0, 0, 1])  # Unit vector in z-direction

# Skew-symmetric function
def skew_symmetric(w):
    return np.array([[0, -w[2], w[1]],
                     [w[2], 0, -w[0]],
                     [-w[1], w[0], 0]])

# Quadrotor Dynamics
def quadrotor_dynamics(t, state):
    x, v, R_flat, omega = np.split(state, [3, 6, 15])
    R = R_flat.reshape((3, 3))

    # Control forces (thrust is slightly tilted to add motion)
    f_thrust = np.array([0.5, 0.2, M * g])  # Slight tilt for forward motion
    tau = np.array([0.01, -0.02, 0.01])  # Small torques

    # Equations of motion
    dxdt = v
    dvdt = -g * e3 + (1/M) * R @ f_thrust
    dRdt = R @ skew_symmetric(omega)
    domegadt = np.linalg.inv(I) @ (tau - np.cross(omega, I @ omega))

    return np.concatenate([dxdt, dvdt, dRdt.flatten(), domegadt])

# Initial Conditions
x0 = np.array([0, 0, 0])  # Initial position
v0 = np.array([0, 0, 0])  # Initial velocity
R0 = np.eye(3).flatten()  # Identity rotation matrix
omega0 = np.array([0, 0, 0])  # Initial angular velocity
state0 = np.concatenate([x0, v0, R0, omega0])

# Time span
t_span = (0, 10)  # Simulate for 10 seconds
t_eval = np.linspace(0, 10, 200)  # Time evaluation points

# Solve ODE
sol = solve_ivp(quadrotor_dynamics, t_span, state0, t_eval=t_eval)

# Extract position data
x_vals = sol.y[0, :]
y_vals = sol.y[1, :]
z_vals = sol.y[2, :]

# Create interactive 3D plot
fig = go.Figure()
fig.add_trace(go.Scatter3d(x=x_vals, y=y_vals, z=z_vals, mode='lines', name='Quadrotor Trajectory', line=dict(color='blue')))
fig.add_trace(go.Scatter3d(x=[x_vals[0]], y=[y_vals[0]], z=[z_vals[0]], mode='markers', marker=dict(color='red', size=6), name='Start Position'))
fig.add_trace(go.Scatter3d(x=[x_vals[-1]], y=[y_vals[-1]], z=[z_vals[-1]], mode='markers', marker=dict(color='green', size=6), name='End Position'))

# Update layout
fig.update_layout(
    title='3D Quadrotor Motion',
    scene=dict(
        xaxis_title='X Position (m)',
        yaxis_title='Y Position (m)',
        zaxis_title='Z Position (m)'
    ),
    template='plotly_white'
)

# Show plot
fig.show()


#Task#3

Deerive a PID controller for the attitude control of a quadrorotor and use simulations to demibstrate the very large region of stability of the controller

Here are few example our own implementations of this controller.

* https://youtu.be/6E9WDQNVSYA
* https://youtu.be/uUKxXImRMOA
* https://youtu.be/zq05N8m_9SA
* https://youtu.be/J5dThZGZN2g
* https://youtu.be/J5MMp6Be3tU
* https://youtu.be/6ZQgE1FI6Wc

In [7]:
import numpy as np
import plotly.graph_objects as go
from scipy.integrate import solve_ivp

M = 1.0
g = 9.81
I = np.diag([0.02, 0.02, 0.04])

kp_pos = np.array([5.0, 5.0, 10.0])
kd_pos = np.array([2.0, 2.0, 5.0])
kp_att = np.array([4.0, 4.0, 8.0])
kd_att = np.array([2.0, 2.0, 4.0])
ki_att = np.array([1.0, 1.0, 2.0])

o_init = np.array([0.0, 0.0, 5.0])
v_init = np.array([0.0, 0.0, 0.0])
R_init = np.eye(3)
omega_init = np.array([0.0, 0.0, 0.0])
error_int = np.zeros(3)

o_target = np.array([0.0, 0.0, 10.0])
R_target = np.eye(3)
omega_target = np.array([0.0, 0.0, 0.0])

T_final = 10.0
dt = 0.01

def skew_symmetric(v):
    return np.array([[0, -v[2], v[1]],
                     [v[2], 0, -v[0]],
                     [-v[1], v[0], 0]])

def pid_attitude_control(error_R, error_omega, error_int):
    return -kp_att * error_R - kd_att * error_omega - ki_att * error_int

def quadrotor_dynamics(t, state):
    global error_int
    o = state[:3]
    v = state[3:6]
    R = state[6:15].reshape(3, 3)
    omega = state[15:18]

    thrust = M * (g + kp_pos[2] * (o_target[2] - o[2]) - kd_pos[2] * v[2])

    error_R = 0.5 * sum(np.cross(R[:, i], R_target[:, i]) for i in range(3))
    error_omega = omega - omega_target
    error_int += error_R * dt
    torque = pid_attitude_control(error_R, error_omega, error_int)

    acc = -g * np.array([0, 0, 1]) + (thrust / M) * (R @ np.array([0, 0, 1]))

    omega_dot = np.linalg.inv(I) @ (torque - np.cross(omega, I @ omega))

    R_dot = skew_symmetric(omega) @ R

    return np.hstack([v, acc, R_dot.flatten(), omega_dot])

state0 = np.hstack([o_init, v_init, R_init.flatten(), omega_init])

time_eval = np.arange(0, T_final, dt)
solution = solve_ivp(quadrotor_dynamics, [0, T_final], state0, t_eval=time_eval, method='RK45')

o_result = solution.y[:3, :].T

fig = go.Figure()
fig.add_trace(go.Scatter(x=solution.t, y=o_result[:, 2], mode='lines', name='Altitude (z)'))
fig.add_trace(go.Scatter(x=solution.t, y=[o_target[2]] * len(solution.t), mode='lines', line=dict(dash='dash', color='red'), name='Target Altitude'))

fig.update_layout(
    title='Quadrotor Altitude Tracking with PID Attitude Control',
    xaxis_title='Time [s]',
    yaxis_title='Altitude [m]',
    legend=dict(x=0.8, y=1.0),
    template='plotly_white'
)

fig.show()


#Task#4

Derive a specialization of the controller develoed in Task#3 for the problem of stabilzing the twin rotor setup in the lab.

In [8]:
import numpy as np
import plotly.graph_objects as go

class PIDController:
    def __init__(self, Kp, Ki, Kd, dt):
        self.Kp = Kp
        self.Ki = Ki
        self.Kd = Kd
        self.dt = dt
        self.integral = 0
        self.previous_error = 0

    def compute(self, error):
        self.integral += error * self.dt
        derivative = (error - self.previous_error) / self.dt
        output = self.Kp * error + self.Ki * self.integral + self.Kd * derivative
        self.previous_error = error
        return output

class TwinRotor:
    def __init__(self, dt=0.01):
        self.dt = dt
        self.angle = np.array([0.0, 0.0])  # [elevation, azimuth]
        self.angular_velocity = np.array([0.0, 0.0])

    def update(self, torque):
        moment_of_inertia = np.array([0.05, 0.05])  # Approximate values
        angular_acceleration = torque / moment_of_inertia
        self.angular_velocity += angular_acceleration * self.dt
        self.angle += self.angular_velocity * self.dt

# Simulation parameters
dt = 0.01  # Time step
time = np.arange(0, 5, dt)  # 5 seconds simulation

# PID controllers for elevation and azimuth
pid_elevation = PIDController(Kp=2.5, Ki=0.15, Kd=1.2, dt=dt)
pid_azimuth = PIDController(Kp=2.0, Ki=0.1, Kd=1.0, dt=dt)

twin_rotor = TwinRotor(dt)

# Desired angles (setpoints)
desired_angle = np.array([10.0, 15.0])  # [elevation, azimuth]

# Data storage
angles = []
for t in time:
    error = desired_angle - twin_rotor.angle
    torque = np.array([
        pid_elevation.compute(error[0]),
        pid_azimuth.compute(error[1])
    ])
    twin_rotor.update(torque)
    angles.append(twin_rotor.angle.copy())

angles = np.array(angles)

# Plot results
fig = go.Figure()
fig.add_trace(go.Scatter(x=time, y=angles[:, 0], mode='lines', name='Elevation (deg)'))
fig.add_trace(go.Scatter(x=time, y=angles[:, 1], mode='lines', name='Azimuth (deg)'))
fig.add_trace(go.Scatter(x=time, y=[desired_angle[0]] * len(time), mode='lines', line=dict(dash='dash', color='blue'), name='Target Elevation'))
fig.add_trace(go.Scatter(x=time, y=[desired_angle[1]] * len(time), mode='lines', line=dict(dash='dash', color='orange'), name='Target Azimuth'))

fig.update_layout(
    title='PID Controlled Twin Rotor Stabilization',
    xaxis_title='Time (s)',
    yaxis_title='Angle (deg)',
    legend=dict(x=0.8, y=1.0),
    template='plotly_white'
)

fig.show()


#Task#5

 Experimentally verify the stability properties of the controller derived in Task#5 using the experimental setup available in the applied mechanics lab.